In [ ]:
pip install networkx numpy

In [ ]:
import networkx as nx
import numpy as np

# BPR link performance function
def bpr_cost(flow, capacity, free_time, alpha=0.15, beta=4):
    return free_time * (1 + alpha * (flow / capacity) ** beta)

# Derivative of the BPR function
def bpr_derivative(flow, capacity, free_time, alpha=0.15, beta=4):
    return free_time * alpha * beta * (flow / capacity) ** (beta - 1) / capacity

# Define a sample network (directed graph)
def create_network():
    G = nx.DiGraph()
    # (from, to, capacity, free flow time)
    links = [
        ('A', 'B', 1000, 1),
        ('A', 'C', 1000, 1),
        ('B', 'D', 1000, 1),
        ('C', 'D', 1000, 1),
    ]
    for u, v, cap, fft in links:
        G.add_edge(u, v, capacity=cap, free_time=fft, flow=0)
    return G

# Update link costs based on current flow
def update_link_costs(G):
    for u, v in G.edges:
        data = G[u][v]
        data['cost'] = bpr_cost(data['flow'], data['capacity'], data['free_time'])

# All-or-nothing assignment
def all_or_nothing(G, od_pairs):
    aux_flow = { (u, v): 0 for u, v in G.edges }
    for origin, dest, demand in od_pairs:
        path = nx.shortest_path(G, origin, dest, weight='cost')
        for i in range(len(path) - 1):
            u, v = path[i], path[i+1]
            aux_flow[(u, v)] += demand
    return aux_flow

# Line search (step size)
def line_search(G, aux_flow):
    numerator, denominator = 0.0, 0.0
    for u, v in G.edges:
        flow = G[u][v]['flow']
        cost = bpr_cost(flow, G[u][v]['capacity'], G[u][v]['free_time'])
        derivative = bpr_derivative(flow, G[u][v]['capacity'], G[u][v]['free_time'])

        delta = aux_flow[(u, v)] - flow
        numerator += cost * delta
        denominator += derivative * delta * delta

    if denominator == 0:
        return 0
    return max(0, min(1, numerator / denominator))

# Update flow
def update_flow(G, aux_flow, step_size):
    for u, v in G.edges:
        G[u][v]['flow'] += step_size * (aux_flow[(u, v)] - G[u][v]['flow'])

# Main Frank-Wolfe algorithm
def frank_wolfe(G, od_pairs, max_iter=50, tol=1e-4):
    for it in range(max_iter):
        update_link_costs(G)
        aux_flow = all_or_nothing(G, od_pairs)
        step_size = line_search(G, aux_flow)
        update_flow(G, aux_flow, step_size)
        gap = sum(abs(aux_flow[(u, v)] - G[u][v]['flow']) for u, v in G.edges)
        print(f"Iteration {it+1}, Gap = {gap:.4f}, Step size = {step_size:.4f}")
        if gap < tol:
            break

# Example usage
if __name__ == "__main__":
    G = create_network()
    od_pairs = [('A', 'D', 1000)]
    frank_wolfe(G, od_pairs)

    print("\nFinal link flows:")
    for u, v in G.edges:
        print(f"{u} -> {v}: Flow = {G[u][v]['flow']:.2f}, Cost = {G[u][v]['cost']:.2f}")

Iteration 1, Gap = 2000.0000, Step size = 0.0000
Iteration 2, Gap = 2000.0000, Step size = 0.0000
Iteration 3, Gap = 2000.0000, Step size = 0.0000
Iteration 4, Gap = 2000.0000, Step size = 0.0000
Iteration 5, Gap = 2000.0000, Step size = 0.0000
Iteration 6, Gap = 2000.0000, Step size = 0.0000
Iteration 7, Gap = 2000.0000, Step size = 0.0000
Iteration 8, Gap = 2000.0000, Step size = 0.0000
Iteration 9, Gap = 2000.0000, Step size = 0.0000
Iteration 10, Gap = 2000.0000, Step size = 0.0000
Iteration 11, Gap = 2000.0000, Step size = 0.0000
Iteration 12, Gap = 2000.0000, Step size = 0.0000
Iteration 13, Gap = 2000.0000, Step size = 0.0000
Iteration 14, Gap = 2000.0000, Step size = 0.0000
Iteration 15, Gap = 2000.0000, Step size = 0.0000
Iteration 16, Gap = 2000.0000, Step size = 0.0000
Iteration 17, Gap = 2000.0000, Step size = 0.0000
Iteration 18, Gap = 2000.0000, Step size = 0.0000
Iteration 19, Gap = 2000.0000, Step size = 0.0000
Iteration 20, Gap = 2000.0000, Step size = 0.0000
Iteration